# Avaliação do Modelo

Esse notebook tem como objetivo a análise de desempenho do modelo vetorial construído anteriormente. Vamos explorar os resultados de forma gráfica e numérica, segmentando a análise em 2 partes principais: uma utilizando o Porter Stemmer no pré-processamento das consultas e da geração de lista invertida, e outra sem essa etapa.

Em um primeiro momento, vamos construir funções generalistas, responsáveis por obter as medidas desejadas. Munidos dessas ferramentas auxiliares, vamos prosseguir para a análise individual comentada anteriormente. 

## Imports e bibliotecas

In [84]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import ast
from sklearn.metrics import classification_report

### Funções de Utilidade

- Carregamento de arquivos

In [85]:
def load_data(filepath, res=False):
    if res == True:
        data = pd.read_csv(f"./results/{filepath}", sep=";", converters={1:ast.literal_eval})
    else:
        data = pd.read_csv(f"./results/{filepath}", sep=";")
    return data     

# Métricas (funções)

- Gráfico de 11 pontos de precisão e recall
- F1-Score
- Precision@5
- Precision@10
- Histograma de R-Precision (comparativo)
- MAP
- MRR
- Discounted Cumulative Gain (médio)
- Normalized Discounted Cumulative Gain 


### 1 - Funções auxiliares

A primeira tríade de funções cobre a obtenção de dados relacionados a uma única consulta

#### 1.1 - Precisão

In [86]:
def get_precision(y_model,y_target):
    
    hits = 0
    for item in y_target:
        if item in y_model:
            hits += 1
    if len(y_model) > 0:
        return hits / len(y_model)
    else:
        return 0

#### 1.2 - Recall

In [87]:
def get_recall(y_model, y_target, vp):
    
    fn = 0
    for item in y_model:
        if item not in y_target:
            fn += 1
    
    if vp + fn != 0:
        return vp/(vp+fn)
    else:
        return 0

#### 1.3 - F1 Score

In [88]:
def f1_score(precision, recall):
    return (2*precision*recall)/(precision+recall)

## Gráfico de precisão e recall (11 pontos)

In [89]:
def plot_precision_and_recall(stats_dict, stem_flag):
    
    recall = [100*stats_dict[1]["recall"]]
    precision = [100*stats_dict[1]["precision"]]

    for i in range(10,101,10):
        recall.append(100*stats_dict[i]["recall"])
        precision.append(100*stats_dict[i]["precision"])


    #plt.plot()
    #plt.xlabel('Recall')
    #plt.ylabel('Precision')
    #plt.title(f'Precisão x Recall ({stem_flag})')

COM STEMMER

# Análise

## Utilizando Stemmer de Porter

In [90]:
stemmed_results = load_data("results-STEMMER.csv",res=True)
stemmed_expected = load_data("expected-STEMMER.csv")


stats_dict = {}
for i in range(1,len(stemmed_expected.index)):

    y_target = stemmed_expected[stemmed_expected['QueryNumber'] == i]['DocNumber'].tolist()
    y_predicted = stemmed_results[stemmed_results['QUERY_ID'] == i]['RESPONSE'].tolist()

    y_model = [row[1] for row in y_predicted if row[2] > 0]
    
    stats_dict[i] = {"precision": get_precision(y_model, y_target)}
    stats_dict[i]["recall"] = get_recall(y_model,y_target, stats_dict[i]["precision"])
    
    
plot_precision_and_recall(stats_dict, "STEMMER")


In [91]:
for query in stats_dict.keys():
    precision, recall = stats_dict[query]["precision"], stats_dict[query]["recall"]
    q_f1 = f1_score(precision, recall)
    stats_dict[query]['f1_score'] = q_f1

stats_df_stem = pd.DataFrame.from_dict(stats_dict)

ZeroDivisionError: float division by zero

## Sem Stemmer de Porter